In [106]:
import sympy as smp
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [107]:
smp.init_printing(use_unicode=True)

In [108]:
class FileSystemManager:
   def __init__(self):
        pass
   def open_file(self, text, filepath, flag):
        """
        Opens a file in read or write mode.
        
        :param text: Text to write to the file if flag is 'w'
        :param filepath: Path of the file to open
        :param flag: 'r' to read, 'w' to write
        :return: File content if reading, None if writing
        """
        try:
            if flag == 'r':
                with open(filepath, 'r') as f:
                    data = f.read()
                    return data  # return instead of print
            elif flag == 'w':
                with open(filepath, 'w') as f:
                    f.write(text)
            else:
                raise ValueError("Unsupported flag. Use 'r' for read or 'w' for write.")
        except FileNotFoundError:
            print(f"Error: File {filepath} not found.")
        except IOError as e:
            print(f"Error accessing file {filepath}: {e}")

In [109]:
class MagmaCalculator:
    """
    A class to interact with the Magma Calculator webpage and submit code for evaluation.
    
    Attributes:
    ----------
    output_file : str
        The name of the file where the output from the Magma Calculator will be saved.
    driver : webdriver.Chrome
        A Chrome web driver instance to automate browser interaction.
    url : str
        The URL of the Magma Calculator page.
    
    Methods:
    -------
    submit_code(code):
        Submits the given Magma code to the calculator and saves the result to a file.
    
    close():
        Closes the browser session.
    """
    
    def __init__(self, output_file="MagmaCalcResult"):
        """
        Initializes the MagmaCalculator with the given output file name.
        
        Parameters:
        ----------
        output_file : str, optional
            The name of the file where the result will be saved (default is "output.txt").
        """
        self.url = "http://magma.maths.usyd.edu.au/calc/"
        self.output_file = output_file
        self.file_manager = FileSystemManager()

    def submit_code(self, code):
        """
        Submits the given code to the Magma Calculator and saves the result to the output file.
        
        Parameters:
        ----------
        code : str
            The Magma code to be submitted for evaluation.
        
        Actions:
        -------
        - Opens the Magma Calculator webpage.
        - Finds the input box and enters the code.
        - Clicks the submit button.
        - Waits for the result to load.
        - Retrieves the result and writes it to the specified output file.
        """
        driver = webdriver.Chrome()
        try:
            driver.get(self.url)
            
            input_box = driver.find_element(By.ID, "input")
            input_box.clear()
            input_box.send_keys(code)
            
            submit_button = driver.find_element(By.XPATH, "//input[@value='Submit']")
            submit_button.click()
            
            time.sleep(5)
            
            result_element = driver.find_element(By.ID, "result")
            result_text = result_element.get_attribute('value')
            
            self.file_manager.open_file(result_text, self.output_file, 'w')
        finally:
            driver.quit()
        


In [110]:
class FlynnLeprevost:
    def __init__(self, m=29, g=2, inf_count=1, curves_path="foundCurves.txt", magma_result_path="MagmaCalcResult"):
        """"""
        self.x, self.y, self.t, self.x0 = smp.symbols('x y t x0')
        self.f = smp.symbols('f')
        #self.u1, self.u2 = smp.symbols('u1 u2')
        self.a1, self.b1, self.a2, self.b2 = smp.symbols('a1 b1 a2 b2')
        self.V1, self.V2, self.W1, self.W2 = smp.symbols('V1 V2 W1 W2') 
        self.yf = smp.Eq(self.y, self.f**2)
        self.m = m
        self.g = g
        if inf_count == 1:
            self.deg = 2 * g + 1
        elif inf_count == 2:
            self.deg = 2 * g + 2
        else:
            raise ValueError("inf_count must be 1 or 2")
        self.constraints = [self.deg, self.deg + self.g]
        self.coeffs = []
        self.curves_path = curves_path
        self.magma_calc = MagmaCalculator(magma_result_path)
        self.file_manager = FileSystemManager()

    def factor_coeffs(self, f, val):
        """
        Factors the coefficients of a polynomial and computes the sum of 
        each factored coefficient multiplied by a power of the given value.
        
        :param f: The polynomial expression whose coefficients will be factored.
        :param val: The symbol or value to use as the base for powers in the sum.
        
        :return: The sum of the factored coefficients of the polynomial `f`,
                each multiplied by the corresponding power of `val`.
                
        Example:
        If f = 2*x**2 + 3*x + 4 and val = x, the function will factor each 
        coefficient (if possible) and return the sum 4 + 3*x + 2*x**2.
        """
        return sum(smp.factor(coeff)*val**i for i, coeff in enumerate(reversed(smp.Poly(f, val).all_coeffs())))

    def involution(self, equation):
        return equation.subs(self.y, -self.y)
    
    def yf_substitute(self, equation):
        return equation.subs(self.y**2, self.f)

    def calculate_determinant(self, expression):
        """
        Calculate the discriminant of a quadratic equation with coefficients a, b, c.
        For a quadratic equation of the form: a*x^2 + b*x + c = 0
        The discriminant D is given by: D = b^2 - 4*a*c
        """
        poly = smp.Poly(expression, self.x)
        a = poly.coeff_monomial(self.x**2)  
        b = poly.coeff_monomial(self.x)      
        c = poly.coeff_monomial(1)     
        D = b**2 - 4 * a * c  
        return smp.factor(D)

    def determinant_full_square(self, D):
        param_eq = smp.Eq(D, self.t**2)
        t_solutions = smp.solve(param_eq, self.t)
        return t_solutions
    
    def factor_quad_eq(self, expression):
        poly = smp.Poly(expression, self.x)
        a = poly.coeff_monomial(self.x**2)  
        b = poly.coeff_monomial(self.x)      
        c = poly.coeff_monomial(1)     
        D = b**2 - 4 * a * c  
        t = self.determinant_full_square(D)[1]
        r1 = (-b + t) / (2 * a)
        r2 = (-b - t) / (2 * a)

        factored_expression_parametrized = a * (self.x - r1) * (self.x - r2)
        return factored_expression_parametrized, t
    
    def is_factorible(self, expression):
        factored_expression = smp.factor(expression)
        if factored_expression.is_Add:
            return False
        else:
            return True

    def calculate_torsion_point_order_divisor(self, a1, b1, a2, b2):
        """Calculate the divisor of torsion point order using the determinant."""
        matrix = smp.Matrix([[a1, b1], [a2, b2]])
        m = abs(matrix.det())
        return m
    
    def check_order(self, equation):
        """Function that helps to check an order of possible torison points of given function"""
        code = f"""
        P<x> := PolynomialRing(Rationals());
        f := {equation};
        f := PolynomialRing(Integers())!f;
        C1 := HyperellipticCurve(f);
        J1 := Jacobian(C1);
        TorsionSubgroup(J1);
        """
        self.magma_calc.submit_code(code, self.magma_calc.output_file)

    def divisor_deg(self, V, W):
        """
        Checks the divisor degree of given function
        deg*: degree of divisor
        deg: degree  of polynomial 
        deg* u_j = max(2 deg(V_j), 2 deg(W_j) + 2g + 1)
        """
        return max(2* smp.degree(V, self.x), 2 * smp.degree(W, self.x) + self.deg)

    def in_constraints(self, coeffs_set):
        return (coeffs_set[0] + coeffs_set[1] <= self.constraints[1]) and (coeffs_set[0] + coeffs_set[1] >= self.constraints[0]) and (coeffs_set[2] + coeffs_set[3] <= self.constraints[1]) and (coeffs_set[2] + coeffs_set[3] >= self.constraints[0])

    def _get_coeffs(self, range_limit=11):
        values = [i for i in range(-range_limit, range_limit)]
        final_coeffs = []
        coeffs_sets = [[a1, b1, a2, b2] for a1 in values for b1 in values for a2 in values for b2 in values]
        for coeffs_set in coeffs_sets:
            if self.calculate_torsion_point_order_divisor(*coeffs_set) == self.m and self.in_constraints(coeffs_set):
                final_coeffs.append(coeffs_set)
        return final_coeffs
    
    def _get_systems(self):
        coeffs_sets = self._get_coeffs(range_limit=11)
        systems = []
        for coeffs_set in coeffs_sets:
            tilda_deg_y = self.deg
            max_deg_V1 = int(np.floor((coeffs_set[0] + coeffs_set[1]) / 2))
            max_deg_V2 = int(np.floor((coeffs_set[2] + coeffs_set[3]) / 2))
            max_deg_W1 = int(np.floor((coeffs_set[0] + coeffs_set[1] - tilda_deg_y) / 2))
            max_deg_W2 = int(np.floor((coeffs_set[2] + coeffs_set[3] - tilda_deg_y) / 2))

            W1 = smp.Poly(sum(smp.symbols(f'd1_{i}') * self.x**i for i in range(max_deg_W1 + 1)), self.x) #  
            W2 = smp.Poly(sum(smp.symbols(f'd2_{i}') * self.x**i for i in range(max_deg_W2 + 1)), self.x) #  

            u1 = self.V1 + W1 * self.y
            u2 = self.V2 + W2 * self.y

            systems.append({
                'coeffs_set': coeffs_set,
                'max_deg_V1': max_deg_V1,
                'max_deg_V2': max_deg_V2,
                'max_deg_W1': max_deg_W1,
                'max_deg_W2': max_deg_W2,
                'W1': W1,
                'W2': W2,
                'u1': u1,
                'u2': u2
            })

        return systems

    def get_final_systems(self):
        final_systems = []
        systems = self._get_systems()
        
        for system in systems:
            u1 = system['u1']
            u2 = system['u2']
            coeffs_set = system['coeffs_set']

            left_part_1 = smp.collect(self.yf_substitute(u1 * self.involution(u1)), self.f)
            left_part_2 = smp.collect(self.yf_substitute(u2 * self.involution(u2)), self.f)

            coeff_1 = -smp.factor(smp.Poly(left_part_1, self.f).coeffs()[0])
            coeff_2 = -smp.factor(smp.Poly(left_part_2, self.f).coeffs()[0])
            
            left_part_1 = self.V1**2 - coeff_1 * self.f
            left_part_2 = self.V2**2 - coeff_2 * self.f

            gamma = smp.symbols('gamma')  
            power_x = smp.symbols('n')     
            power_x_minus_1 = smp.symbols('m')  

            equation = gamma * self.x**power_x * (self.x - 1)**power_x_minus_1 

            right_part_1 = equation.subs({power_x: coeffs_set[0], power_x_minus_1: coeffs_set[1]})
            right_part_2 = equation.subs({power_x: coeffs_set[2], power_x_minus_1: coeffs_set[3]})

            eq_1 = smp.Eq(left_part_1, right_part_1)
            eq_2 = smp.Eq(left_part_2, right_part_2)

            system_eqs = {
                'coeffs_set': system['coeffs_set'],
                'max_deg_V1': system['max_deg_V1'],
                'max_deg_V2': system['max_deg_V2'],
                'max_deg_W1': system['max_deg_W1'],
                'max_deg_W2': system['max_deg_W2'],
                'left_part_1': left_part_1,
                'left_part_2': left_part_2,
                'right_part_1': right_part_1,
                'right_part_2': right_part_2,
                'eq_1': eq_1,
                'eq_2': eq_2,
                'coeff_1': coeff_1,
                'coeff_2': coeff_2
            }
            
            final_systems.append(system_eqs)
    
        return final_systems
    
    def get_simpyified_equation(self):
        final_equations = []
        roots_glob = []
        systems = self.get_final_systems()
        
        for system in systems:
            coeff_1 = system['coeff_1']
            coeff_2 = system['coeff_2']
            
            max_deg_V1 = system['max_deg_V1']
            max_deg_V2 = system['max_deg_V2']

            right_part_1 = system['right_part_1']
            right_part_2 = system['right_part_2']

            left_part = smp.factor(self.V1**2 * coeff_2 - self.V2**2 * coeff_1, self.V1, self.V2)
            right_part = smp.factor(right_part_1 * coeff_2 - right_part_2 * coeff_1, self.x)

            right_part_degree = smp.degree(right_part, self.x)
            left_part_degree = smp.degree(self.x**(2*max_deg_V1) * coeff_2 - self.x**(2*max_deg_V2) * coeff_1, self.x)

            degree_spread = right_part_degree - left_part_degree

            if degree_spread > 0:
                for deg in range(right_part_degree, left_part_degree, -1):
                    term_coeff = smp.Poly(right_part, self.x).coeff_monomial(self.x**deg)
                    right_part -= term_coeff * self.x**deg

            right_part = smp.factor(right_part, self.x)
            
            roots = smp.roots(smp.Poly(right_part, self.x))
            roots_glob.append(roots)

            factorized = smp.Mul(*[(self.x - root)**multiplicity for root, multiplicity in roots.items()])

            final_equation = smp.Eq(left_part, factorized)
            final_equations.append(final_equation)

        answer = {
            'final_equations': final_equations,
            'roots': roots_glob,
            'systems': systems
        }
            
        return answer

In [111]:
model = FlynnLeprevost()

In [112]:
coeffs = model._get_coeffs()

In [113]:
coeffs

[[-4, 9, 1, 5], [-3, 10, 2, 3], [-2, 7, 3, 4], [-1, 7, 4, 1], [1, 4, 7, -1], [ ↪

↪ 1, 5, -4, 9], [1, 5, 6, 1], [1, 6, 5, 1], [2, 3, -3, 10], [3, 2, 10, -3], [3 ↪

↪ , 4, -2, 7], [4, 1, -1, 7], [4, 3, 7, -2], [5, 1, 1, 6], [5, 1, 9, -4], [6,  ↪

↪ 1, 1, 5], [7, -2, 4, 3], [7, -1, 1, 4], [9, -4, 5, 1], [10, -3, 3, 2]]

In [114]:
systems = model._get_systems()

In [115]:
systems

[{'coeffs_set': [-4, 9, 1, 5],
  'max_deg_V1': 2,
  'max_deg_V2': 3,
  'max_deg_W1': 0,
  'max_deg_W2': 0,
  'W1': Poly(d1_0, x, domain='ZZ[d1_0]'),
  'W2': Poly(d2_0, x, domain='ZZ[d2_0]'),
  'u1': Poly(V1 + d1_0*y, x, domain='ZZ[V1,d1_0,y]'),
  'u2': Poly(V2 + d2_0*y, x, domain='ZZ[V2,d2_0,y]')},
 {'coeffs_set': [-3, 10, 2, 3],
  'max_deg_V1': 3,
  'max_deg_V2': 2,
  'max_deg_W1': 1,
  'max_deg_W2': 0,
  'W1': Poly(d1_1*x + d1_0, x, domain='ZZ[d1_0,d1_1]'),
  'W2': Poly(d2_0, x, domain='ZZ[d2_0]'),
  'u1': Poly(d1_1*y*x + V1 + d1_0*y, x, domain='ZZ[V1,d1_0,d1_1,y]'),
  'u2': Poly(V2 + d2_0*y, x, domain='ZZ[V2,d2_0,y]')},
 {'coeffs_set': [-2, 7, 3, 4],
  'max_deg_V1': 2,
  'max_deg_V2': 3,
  'max_deg_W1': 0,
  'max_deg_W2': 1,
  'W1': Poly(d1_0, x, domain='ZZ[d1_0]'),
  'W2': Poly(d2_1*x + d2_0, x, domain='ZZ[d2_0,d2_1]'),
  'u1': Poly(V1 + d1_0*y, x, domain='ZZ[V1,d1_0,y]'),
  'u2': Poly(d2_1*y*x + V2 + d2_0*y, x, domain='ZZ[V2,d2_0,d2_1,y]')},
 {'coeffs_set': [-1, 7, 4, 1],
  'max_d

In [116]:
systems = model.get_final_systems()

In [117]:
systems

[{'coeffs_set': [-4, 9, 1, 5],
  'max_deg_V1': 2,
  'max_deg_V2': 3,
  'max_deg_W1': 0,
  'max_deg_W2': 0,
  'left_part_1': V1**2 - d1_0**2*f,
  'left_part_2': V2**2 - d2_0**2*f,
  'right_part_1': gamma*(x - 1)**9/x**4,
  'right_part_2': gamma*x*(x - 1)**5,
  'eq_1': Eq(V1**2 - d1_0**2*f, gamma*(x - 1)**9/x**4),
  'eq_2': Eq(V2**2 - d2_0**2*f, gamma*x*(x - 1)**5),
  'coeff_1': d1_0**2,
  'coeff_2': d2_0**2},
 {'coeffs_set': [-3, 10, 2, 3],
  'max_deg_V1': 3,
  'max_deg_V2': 2,
  'max_deg_W1': 1,
  'max_deg_W2': 0,
  'left_part_1': V1**2 - f*(d1_0 + d1_1*x)**2,
  'left_part_2': V2**2 - d2_0**2*f,
  'right_part_1': gamma*(x - 1)**10/x**3,
  'right_part_2': gamma*x**2*(x - 1)**3,
  'eq_1': Eq(V1**2 - f*(d1_0 + d1_1*x)**2, gamma*(x - 1)**10/x**3),
  'eq_2': Eq(V2**2 - d2_0**2*f, gamma*x**2*(x - 1)**3),
  'coeff_1': (d1_0 + d1_1*x)**2,
  'coeff_2': d2_0**2},
 {'coeffs_set': [-2, 7, 3, 4],
  'max_deg_V1': 2,
  'max_deg_V2': 3,
  'max_deg_W1': 0,
  'max_deg_W2': 1,
  'left_part_1': V1**2 - d1

In [118]:
current_system = systems[1]

In [119]:
current_system

{'coeffs_set': [-3, 10, 2, 3],
 'max_deg_V1': 3,
 'max_deg_V2': 2,
 'max_deg_W1': 1,
 'max_deg_W2': 0,
 'left_part_1': V1**2 - f*(d1_0 + d1_1*x)**2,
 'left_part_2': V2**2 - d2_0**2*f,
 'right_part_1': gamma*(x - 1)**10/x**3,
 'right_part_2': gamma*x**2*(x - 1)**3,
 'eq_1': Eq(V1**2 - f*(d1_0 + d1_1*x)**2, gamma*(x - 1)**10/x**3),
 'eq_2': Eq(V2**2 - d2_0**2*f, gamma*x**2*(x - 1)**3),
 'coeff_1': (d1_0 + d1_1*x)**2,
 'coeff_2': d2_0**2}

In [120]:
current_system['eq_1']

                                    10
  2                    2   γ⋅(x - 1)  
V₁  - f⋅(d₁ ₀ + d₁ ₁⋅x)  = ───────────
                                3     
                               x      

In [121]:
current_system['eq_2']

  2       2        2        3
V₂  - d₂ ₀ ⋅f = γ⋅x ⋅(x - 1) 

In [ ]:
equations = model.get_simpyified_equation()

In [ ]:
equations['systems'][0]

{'coeffs_set': [1, 5, 6, 1],
 'max_deg_V1': 3,
 'max_deg_V2': 3,
 'max_deg_W1': 0,
 'max_deg_W2': 1,
 'left_part_1': V1**2 - d1_0**2*f,
 'left_part_2': V2**2 - f*(d2_0 + d2_1*x)**2,
 'right_part_1': gamma*x*(x - 1)**5,
 'right_part_2': gamma*x**6*(x - 1),
 'eq_1': Eq(V1**2 - d1_0**2*f, gamma*x*(x - 1)**5),
 'eq_2': Eq(V2**2 - f*(d2_0 + d2_1*x)**2, gamma*x**6*(x - 1)),
 'coeff_1': d1_0**2,
 'coeff_2': (d2_0 + d2_1*x)**2}

In [ ]:
equations['roots'][0]

{0: 1, 1: 1}

In [ ]:
equations['final_equations'][0]

(V₁⋅(d₂ ₀ + d₂ ₁⋅x) - V₂⋅d₁ ₀)⋅(V₁⋅(d₂ ₀ + d₂ ₁⋅x) + V₂⋅d₁ ₀) = x⋅(x - 1)

In [ ]:
code =[ """
    Z := Integers();
    p := Factorial(10);
    p;
""",
"""
    P<x> := PolynomialRing(Rationals());
    f := 15876 * 21 * 7 * (50*x^5 - 1289375/15876*x^4 - 2075/21*x^3 + 13800/49*x^2 - 1350/7*x + 2025/49);
    f := PolynomialRing(Integers())!f;
    C1 := HyperellipticCurve(f);
    J1 := Jacobian(C1);
    TorsionSubgroup(J1);
    """
    ]

magma_calc = MagmaCalculator()
# magma_calc.submit_code(code[1])


In [ ]:
x, x0, t, m, n = smp.symbols('x x0 t m n')
expression = (2 * x0 + 1) * x**2 - x0 * (x0 + 2) * x + x0**2

D = model.calculate_determinant(expression)
t_solution = model.determinant_full_square(D)
t_solution

⎡    ______________     ______________⎤
⎢   ╱   3              ╱   3          ⎥
⎣-╲╱  x₀ ⋅(x₀ - 4) , ╲╱  x₀ ⋅(x₀ - 4) ⎦

In [ ]:
def factor_quad_eq(expression):
    poly = smp.Poly(expression, x)
    a = poly.coeff_monomial(x**2)  
    b = poly.coeff_monomial(x)      
    c = poly.coeff_monomial(1)     
    D = b**2 - 4 * a * c  
    t = model.determinant_full_square(D)[1]
    r1 = (-b + t) / (2 * a)
    r2 = (-b - t) / (2 * a)

    factored_expression_parametrized = a * (x - r1) * (x - r2)
    return factored_expression_parametrized, t
    

In [ ]:
expression = (2 * x0 + 1) * x**2 - x0 * (x0 + 2) * x + x0**2
factor_quad_eq(expression)

⎛⎛                    ______________⎞ ⎛                    ______________⎞     ↪
⎜⎜      2            ╱   3          ⎟ ⎜      2            ╱   3          ⎟     ↪
⎜⎜    x₀  + 2⋅x₀ - ╲╱  x₀ ⋅(x₀ - 4) ⎟ ⎜    x₀  + 2⋅x₀ + ╲╱  x₀ ⋅(x₀ - 4) ⎟     ↪
⎜⎜x - ──────────────────────────────⎟⋅⎜x - ──────────────────────────────⎟⋅(2⋅ ↪
⎝⎝               4⋅x₀ + 2           ⎠ ⎝               4⋅x₀ + 2           ⎠     ↪

↪                           ⎞
↪             ______________⎟
↪            ╱   3          ⎟
↪ x₀ + 1), ╲╱  x₀ ⋅(x₀ - 4) ⎟
↪                           ⎠

In [ ]:
D_parametrized = (m**2 + n**2)**2

param_eq_pythagorean = smp.Eq(D, D_parametrized)

x0_solution = smp.solve(param_eq_pythagorean, x0)
x0_solution[0]


⎧                                                                              ↪
⎪                                                                              ↪
⎪                                                                              ↪
⎪                                                                              ↪
⎪                                                                              ↪
⎪                                                                              ↪
⎪                                                                              ↪
⎪                                                                              ↪
⎪                                                                              ↪
⎪                                                                              ↪
⎪                                                                              ↪
⎪                                                                              ↪
⎪                           

In [ ]:
from sympy import symbols, Eq, simplify

# Define symbols
m, n = symbols('m n', integer=True)

# Define the Pythagorean parametrization
x = m**2 - n**2
y = 2 * m * n
z = m**2 + n**2

# Verify the Pythagorean theorem
equation = Eq(x**2 + y**2, z**2)
verification = simplify(x**2 + y**2 - z**2)

# Print results
print(f"x = {x}")
print(f"y = {y}")
print(f"z = {z}")
print(f"Verification: {verification == 0} ({verification})")

x = m**2 - n**2
y = 2*m*n
z = m**2 + n**2
Verification: True (0)


In [ ]:
from sympy import symbols, Eq, solve, simplify, prod

x = symbols('x')  # Переменная для многочлена
a, b, c = symbols('a b c', rational=True)  # Параметры

roots = [a, b + 1/2, c - 1]  # Корни
multiplicities = [2, 1, 3]  # Кратности

polynomial = simplify(prod([(x - root)**multiplicity for root, multiplicity in zip(roots, multiplicities)]))

conditions = [Eq(simplify(root), root) for root in roots]

solutions = solve(conditions, [a, b, c], domain='QQ')

print("Полином:", polynomial)
print("Условия рациональности параметров:", solutions)

Полином: (a - x)**2*(-b + x - 0.5)*(-c + x + 1)**3
Условия рациональности параметров: []


In [ ]:
from sympy import symbols, Eq

# Define symbols
x = symbols('x')
a, b, c = symbols('a b c')

# Define an equation
equation = Eq(a * x**2 + b * x + c, 0)

# Substitute specific values for parameters
specific_values = {a: 2, b: 3, c: 4}
new_equation = equation.subs(specific_values)

# Output the new equation
print("Original equation:", equation)
print("Equation after substitution:", new_equation)


Original equation: Eq(a*x**2 + b*x + c, 0)
Equation after substitution: Eq(2*x**2 + 3*x + 4, 0)


In [ ]:
from sympy import symbols, expand, Eq, solve

# Define variables
x, V1, V2, d10, d11, d20, d21, d22 = symbols('x V1 V2 d10 d11 d20 d21 d22')

# Define the equation
lhs = (V1 * (d20 + d21 * x + d22 * x**2) + V2 * (-d10 - d11 * x)) * (V1 * (d20 + d21 * x + d22 * x**2) + V2 * (d10 + d11 * x))
rhs = x**5 * (x - 1)**8
equation = Eq(lhs, rhs)

# Expand the left-hand side to extract the factors
lhs_factors = expand(lhs).args  # Extract terms (factors) of multiplication
rhs_factors = [x**5, (x - 1)**8]  # Right-hand side factors

# Equate corresponding factors
first_equation = Eq(lhs_factors[0], rhs_factors[0])  # First bracket to first factor
second_equation = Eq(lhs_factors[1], rhs_factors[1])  # Second bracket to second factor

# Output the equations
print("First equation:", first_equation)
print("Second equation:", second_equation)

# Solve the equations if needed
solution_1 = solve(first_equation, V1)
solution_2 = solve(second_equation, V2)

print("Solutions for first equation:", solution_1)
print("Solutions for second equation:", solution_2)

First equation: Eq(V1**2*d20**2, x**5)
Second equation: Eq(-V2**2*d10**2, (x - 1)**8)
Solutions for first equation: [-sqrt(x**5)/d20, sqrt(x**5)/d20]
Solutions for second equation: [-I*(x - 1)**4/d10, I*(x - 1)**4/d10]
